## Fixing the issue with losses not matching in 2 codes


In [6]:
import numpy as np
import matplotlib.pyplot as plt
# import pandas as pd

import subprocess as sp
import sys
import      os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize

import pandas as pd

import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torchsummary import summary

import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, TensorDataset

import time
from datetime import datetime
import glob
import pickle
import yaml

from scipy import fftpack

In [7]:
%matplotlib widget

In [8]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/cosmogan/1_main_code/')
import spec_loss as spc
import post_analysis_pandas as post

In [9]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)



## Read data

In [10]:
ip_fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
img=np.load(ip_fname,mmap_mode='r')[:1000].transpose(0,1,2,3)
t_img=torch.from_numpy(img)
img.shape,t_img.shape

((1000, 1, 128, 128), torch.Size([1000, 1, 128, 128]))

In [11]:
ls

1_pytorch_analyze-results.ipynb
2_analyze_pytorch_without_precompute-.ipynb
3_pytorch_compare_data_keras_pytorch.ipynb
5_Debug_loss.ipynb
6_test_spectra_pytorch.ipynb
6_test_spectra_tensorflow.ipynb
lbann_3c_find_best_epochs.ipynb
lbann_4b_lbann_analyze_chisqrs.ipynb


## Spectrum check

Link to use fftpack for pytorch : https://github.com/locuslab/pytorch_fft/issues/9


In [ ]:
# mean,sdev=f_torch_image_spectrum(f_invtransform(fake),1,r.to(device),ind.to(device))
r,ind=spc.f_get_rad(img)
mean,sdev=spc.f_torch_image_spectrum(t_img,1,r,ind)

dict_sample=post.f_compute_hist_spect(img[:,0,:,:],bins=50)

In [ ]:
dict_sample['spec_val'].shape,mean[0].shape

In [ ]:
### Test mean 
ans1=mean[0,:]
ans2=dict_sample['spec_val']

for i in range(len(ans2)):
    a,b=ans1[i].item(),ans2[i]
    if ((a-b)>1e-2):
        print(i,a,b)


In [ ]:
### Test stderr
ans1=sdev[0,:]
ans2=dict_sample['spec_err']

for i in range(len(ans2)):
    a,b=ans1[i].item(),ans2[i]
    if ((a-b)>1e-2):
        print(i,a,b)

### Inference: 
Oct 22, 2020 (Bday!) \
The spectra match. \
The stderr matches less, but it's still in the ballpark. deviation probably due to rounding off errors.

Oct 29, 2022. \
Modified pytorch radial profile code to remove bincount to help with backprop. code taken from tensorflow. \
Results agree!

### Check loss functions

In [35]:
ip_fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
img=np.load(ip_fname,mmap_mode='r')[-3000:].transpose(0,1,2,3)
print(img.shape)

(3000, 1, 128, 128)


In [36]:
f1='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/exagan1/run5_fixed_cosmology/models/gen_imgs.npy'
f1='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201110_072035_full_loss_b64_with-invtransform/images/gen_img_epoch-14_step-44862.npy'
img_1=np.expand_dims(np.load(f1)[-3000:],axis=1)

# f2=''
# img_2=np.expand_dims(np.load(f2)[:1000],axis=1)

# ip_fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
# img_1=np.load(ip_fname,mmap_mode='r')[5000:6000].transpose(0,1,2,3)

print(img_1.shape)
# img_1=f_invtransform(img_1)

(64, 1, 128, 128)


In [ ]:
# Loss from pytorch training code
r,ind=spc.f_get_rad(img)
mean_spec_data,sdev_spec_data=spc.f_torch_image_spectrum(torch.from_numpy(img),1,r,ind)
m1,s1=spc.f_torch_image_spectrum(torch.from_numpy(img_1),1,r,ind)
print(spc.loss_spectrum(m1,mean_spec_data,s1,sdev_spec_data,128))


In [ ]:
# Loss from post_analysis code
dict_val=post.f_compute_hist_spect(img[:,0,:,:],bins=50)
dict_sample=post.f_compute_hist_spect(img_1[:,0,:,:],bins=50)
post.f_compute_chisqr(dict_val,dict_sample)['chi_spec3']

In [12]:
# m1,mean_spec_data

### Inference:
Nov 6, 2020.
Added a new spectrum metric 'chi_spec3' that computes the loss.
The results match very well !


## Histogram check

In [ ]:
def f_batch_histogram(img_arr,bins,norm,hist_range):
    ''' Compute histogram statistics for a batch of images'''

    ## Extracting the range. This is important to ensure that the different histograms are compared correctly
    if hist_range==None : ulim,llim=np.max(img_arr),np.min(img_arr)
    else: ulim,llim=hist_range[1],hist_range[0]
#         print(ulim,llim)
    ### array of histogram of each image
    hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important
    hist=np.stack(hist_arr[:,0]) # First element is histogram array
#         print(hist.shape)
    bin_list=np.stack(hist_arr[:,1]) # Second element is bin value 
    ### Compute statistics over histograms of individual images
    mean,err=np.mean(hist,axis=0),np.std(hist,axis=0)/np.sqrt(hist.shape[0])
    bin_edges=bin_list[0]
    centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    return mean,err,centers


In [ ]:
def f_compute_hist(data,bins):
    hist_data=torch.histc(data,bins=bins)
    hist_data=(hist_data*bins)/torch.sum(hist_data)
    
    return hist_data

a=f_compute_hist(torch.from_numpy(img),10)
b=f_batch_histogram(img,10,True,None)[0]

print(a,b)

In [ ]:
plt.figure()
plt.plot(a,linestyle='',marker='*')
plt.plot(b,linestyle='',marker='o')

### Test the loss functions on results and keras results

In [ ]:
f1='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/exagan1/run5_fixed_cosmology/models/gen_imgs.npy'
img_1=np.expand_dims(np.load(f1)[:500],axis=1)

f2=''
img_2=np.expand_dims(np.load(f2)[:1000],axis=1)


In [ ]:
r,ind=spc.f_get_rad(img)
mean_spec_data,sdev_spec_data=spc.f_torch_image_spectrum(t_img[:1000],1,r,ind)
m1,s1=spc.f_torch_image_spectrum(torch.from_numpy(img_1),1,r,ind)
m2,s2=spc.f_torch_image_spectrum(torch.from_numpy(img_2),1,r,ind)

In [ ]:
print(spc.loss_spectrum(m1,mean_spec_data,s1,sdev_spec_data,128))
print(spc.loss_spectrum(m2,mean_spec_data,s2,sdev_spec_data,128))

In [ ]:
hist_data=torch.histc(t_img[:1000],bins=50)
print(spc.loss_hist(t_img[5000:7000],hist_data))
print(spc.loss_hist(torch.from_numpy(img_1),hist_data),spc.loss_hist(torch.from_numpy(img_2),hist_data))

### Inference
Oct 20, 2020
There are variations in the spectrum functions in pytorch and numpy.
1. fftshit
2. abs()**2

However, the loss is computed with the same function for input and generated images, so it should not matter.
The comparison with pytorch and keras data shows that keras is doing better (for some reason).

Nov 6, 2020
Modifications have been made to incorporate the variations mentioned above.
But, keras still better...